In [4]:
from keras.models import Model
from keras.layers import Input, Conv3D, MaxPooling3D, concatenate, Conv3DTranspose, BatchNormalization, Dropout, Lambda
from keras.optimizers import Adam
from keras.metrics import MeanIoU

2022-12-31 14:30:23.938042: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-31 14:30:24.486917: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-31 14:30:24.847943: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-31 14:30:24.847988: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [6]:
import tensorflow as tf
tf.__version__

'2.11.0'

In [7]:
kernel_initializer =  'he_uniform' #Try others if you want

def down_block(input, kernels, drop_ratio=0):
    conv = Conv3D(kernels, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(input)
    d = Dropout(drop_ratio)(conv)
    conv = Conv3D(kernels, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(d)
    p = MaxPooling3D((2, 2, 2))(conv)
    return (p,conv)

def up_block(input, kernels, drop_ratio, concat_layer):
    conv = Conv3DTranspose(kernels, (2, 2, 2), strides=(2, 2, 2), padding='same')(input)
    concat = concatenate([conv, concat_layer])
    conv = Conv3D(kernels, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(concat)
    d = Dropout(drop_ratio)(conv)
    conv = Conv3D(kernels, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(d)

    return conv
  

def UNET_3D(IMG_HEIGHT, IMG_WIDTH, IMG_DEPTH, IMG_CHANNELS, num_classes):
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_DEPTH, IMG_CHANNELS))
    #s = Lambda(lambda x: x / 255)(inputs)   #No need for this if we normalize our inputs beforehand
    s = inputs

    #Contraction path
    (p1,c1) = down_block(s, 16, 0.1)
    (p2,c2) = down_block(p1, 32, 0.1)
    (p3,c3) = down_block(p2, 64, 0.2)
    (p4,c4) = down_block(p3, 128, 0.2)

    c5 = Conv3D(256, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(p4)
    c5 = Dropout(0.3)(c5)
    c5 = Conv3D(256, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c5)

    #Expansive path 
    c6 = up_block(c5, 128, 0.2, c4)
    c7 = up_block(c6, 64, 0.2, c3)
    c8 = up_block(c7, 32, 0.1, c2)
    c9 = up_block(c8, 16, 0.1, c1)

    outputs = Conv3D(num_classes, (1, 1, 1), activation='softmax')(c9)
    model = Model(inputs=[inputs], outputs=[outputs])
    #compile model outside of this function to make it flexible. 
    # model.summary()

    return model

#Test if everything is working ok. 
model = UNET_3D(128, 128, 128, 3, 4)
print(model.input_shape)
print(model.output_shape)

2022-12-31 14:31:14.213261: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-31 14:31:14.213299: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-31 14:31:14.213324: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (omm-Vostro-3491): /proc/driver/nvidia/version does not exist
2022-12-31 14:31:14.213703: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


(None, 128, 128, 128, 3)
(None, 128, 128, 128, 4)
